In [2]:
#Project: The program takes in the data processed by DataCollection
#and performs the calculations necessary to predict a winner
#CSE 482 Term Project
#Ian Murray
#Keaton Coffman

#---Static Details---
import pandas as p
import ipywidgets as widgets

baseWinChance = 0.50 #The default chance a team has to win
homeCourtAdvantage = 0.05 #The % advantage the home team gets
PPPOffenseScale = 1.5 #Weight for offensive prowess
PPPDefenseScale = 1.5 #Weight for defensive tenacity
WinRecordScale = 0.25 #Weight for win/loss reocord
ScoreScale = 0.25 #Weight for how much score deviates from % win chance

In [3]:
#---Format input data into one DataFrame---

#Read in offensive, defensive, strength-of-schedule data for each team
record = p.read_csv("SOS.csv", header="infer")
dataOffense = p.read_csv("offense.csv", header = "infer")
dataDefense = p.read_csv("defense.csv", header = "infer")

#Merge the three previous DataFrames for each team
data = p.merge(dataOffense, dataDefense, on='Team')
data = p.merge(data, record, on='Team')

#Drop 'League Average' and index data by team, not arbitrary rank
data = data[data["Team"] != "League Average"]
data.set_index("Team", inplace = True)

In [7]:
#---Perform Calculations---
#home = ["Los Angeles Lakers", "Detroit Pistons", "Memphis Grizzlies", "Brooklyn Nets", "Toronto Raptors", "Atlanta Hawks", "Sacramento Kings", "Portland Trail Blazers"]
#away = ["Orlando Magic", "Phoenix Suns", "New York Knicks", "Philadelphia 76ers", "Miami Heat", "Charlotte Hornets", "Utah Jazz", "Los Angeles Clippers"]
import ipywidgets as widgets
from ipywidgets import Dropdown
from IPython.display import clear_output

#home = ["Los Angeles Lakers", "Detroit Pistons", "Memphis Grizzlies", "Brooklyn Nets", "Toronto Raptors", "Atlanta Hawks", "Sacramento Kings", "Portland Trail Blazers"]
#away = ["Orlando Magic", "Phoenix Suns", "New York Knicks", "Philadelphia 76ers", "Miami Heat", "Charlotte Hornets", "Utah Jazz", "Los Angeles Clippers"]
home ='Detroit Pistons'
away ='Orlando Magic'

teams= ['Atlanta Hawks','Boston Celtics','Cleveland Cavaliers',
            'New Orleans Pelicans','Chicago Bulls','Dallas Mavericks',
            'Denver Nuggets','Golden State Warriors','Houston Rockets',
            'Los Angeles Clippers','Los Angeles Lakers','Miami Heat',
            'Milwaukee Bucks', 'Minnesota Timberwolves','Brooklyn Nets',
            'New York Knicks', 'Orlando Magic', 'Indiana Pacers',
            'Philadelphia 76ers','Phoenix Suns','Portland Trail Blazers',
            'Sacramento Kings','San Antonio Spurs','Oklahoma City Thunder',
            'Toronto Raptors','Utah Jazz', 'Memphis Grizzlies',
            'Washington Wizards','Detroit Pistons','Charlotte Hornets']

teams.sort()

w = widgets.Dropdown(
    options= teams,
    value='Detroit Pistons',
    description='Home Team',
    disabled=False,
)


y = widgets.Dropdown(
    options=teams,
    value='Orlando Magic',
    description='Away Team',
    disabled=False,
)

display(w)
display(y)

def handle_change(change):
    home = w.value
    away =  y.value
    display_results(home,away)


def display_results(home, away):
#for k in range(0, len(home)):
    #Calculate win chances for each team.
    clear_output(wait=True)
    home = home
    away = away
    winChanceHome = baseWinChance + homeCourtAdvantage
    winChanceHome += PPPOffenseScale * (float(data.loc[home]["PPPOffense"]) - float(data.loc[away]["PPPOffense"]))
    winChanceHome += PPPDefenseScale * ((1 / float(data.loc[home]["PPPDefense"])) - (1 / float(data.loc[away]["PPPDefense"])))
    winChanceHome += WinRecordScale * ((data.loc[home]['WLRatio'] * data.loc[home]['SOS']) - (data.loc[away]['WLRatio'] * data.loc[away]['SOS']))
    winChanceAway = 1 - winChanceHome

    #Calculate predicted score
    homeScore = int(data.loc[home]["PPG"] * (1 - (ScoreScale * (baseWinChance - winChanceHome))))
    awayScore = int(data.loc[away]["PPG"] * (1 - (ScoreScale * (baseWinChance - winChanceAway))))

    #Convert and set percentage boundaries for edge cases
    winChanceHome *= 100
    if winChanceHome > 100:
        winChanceHome = 100
    if winChanceHome < 0:
        winChanceHome = 0
    if homeScore == awayScore:
        homeScore += 1
    display(w)
    display(y)
    #Output
    return print("Home: " + home + ", Score: " + str(homeScore) +
          "\nAway: " + away + ", Score: " + str(awayScore) + 
          "\n" + home + " Win Odds: " + str(round(winChanceHome)) + "%\n")

w.observe(handle_change,names="value")
y.observe(handle_change,names="value")

Dropdown(description='Home Team', index=8, options=('Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets', 'Charl…

Dropdown(description='Away Team', index=21, options=('Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets', 'Char…